# Demonstrate visibility xarray format

In [ ]:
import astropy.units as u
import numpy
from astropy.coordinates import SkyCoord

from rascil.data_models import PolarisationFrame
from rascil.processing_components.simulation import create_named_configuration
from rascil.processing_components.visibility.base import create_blockvisibility

Set up a test visibility set

In [ ]:
lowcore = create_named_configuration('LOWBD2-CORE', rmax=1000)
times = (numpy.pi / 43200.0) * numpy.arange(-4 * 3600, +4 * 3600.0, 1800)
frequency = numpy.linspace(1.0e8, 1.1e8, 5)
channel_bandwidth = numpy.array([1e7, 1e7, 1e7, 1e7, 1e7])
# Define the component and give it some spectral behaviour
f = numpy.array([100.0, 20.0, -10.0, 1.0])
flux = numpy.array([f, 0.8 * f, 0.6 * f, 0.4 * f, 0.2 * f])
phasecentre = SkyCoord(ra=+180.0 * u.deg, dec=-35.0 * u.deg, frame='icrs',
                       equinox='J2000')
xvis = create_blockvisibility(lowcore, times, frequency,
                                  channel_bandwidth=channel_bandwidth,
                                  phasecentre=phasecentre,
                                  integration_time=1800.0,
                                  polarisation_frame=PolarisationFrame("linear"),
                                  weight=1.0)

In [ ]:
assert xvis['vis'].shape == (16, 13861, 5, 4), xvis['vis'].shape
assert xvis["uvw"].data.shape == (16, 13861, 3), xvis["uvw"].shape
assert xvis["uvw_lambda"].data.shape == (16, 13861, 5, 3), xvis["uvw_lambda"].data.shape

Initial xvis

In [ ]:
print(xvis)

Selection of the Dataset by polarisation

In [ ]:
print(xvis.sel({"polarisation": ["XY", "YX"]}))


Selection of the Dataset by frequency

In [ ]:
print(xvis.sel({"frequency": [1.0e8, 1.1e8]}))


sel antenna1 yields smaller XVisibility

In [ ]:
print(xvis.sel(indexers={"antenna1":10}, drop=False))

where antenna1 yields masked arrays

In [ ]:
print(xvis.where(xvis["antenna1"] == 10), )

By uvdist yields masked arrays

In [ ]:
print(xvis.where(xvis.uvdist_lambda < 40.0))

By time

In [ ]:
print(xvis.where(xvis["datetime"] > numpy.datetime64("2020-01-01T23:00:00")))

Grouped by datetime

In [ ]:
for result in xvis.groupby("datetime"):
    print(result[0], result[1].dims)

In bins of antenna1

In [ ]:
for result in xvis.groupby_bins(xvis["antenna1"], bins=16):
    print(result[0], result[1].dims)

Set values using the loc operator

In [ ]:
xvis['vis'].loc[{"polarisation":["XY", "YX"]}] = 0.0
xvis['vis'].loc[{"polarisation":["XX", "YY"], "frequency":[1.1e8]}] = 1.0
xvis['vis'].loc[{"polarisation":["XX", "YY"], "frequency":[1.1e8], "antenna1":slice(100,110)}] = 1.0

In [ ]:
print(xvis['vis'].argmax(("time",)))

In [ ]:
from rascil.processing_components import plot_uvcoverage
plot_uvcoverage([xvis])

Recast as Visibility using an unstack of the baseline and then a stack of the various dimensions

In [ ]:
xtaaf=xvis.unstack("baselines").stack(taaf_index=["time", "antenna2", "antenna1", "frequency"])
print(xtaaf.uvw)
print(xtaaf.vis)